In [2]:
import torch
from torchvision import datasets, transforms, utils

## Data

In [3]:
# Define transformation (e.g., convert to tensor)
transform = transforms.ToTensor()

# Download and load training and test datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Combine train and test datasets
dataset = torch.utils.data.ConcatDataset([train_dataset, test_dataset])

ind = [i for i in range(len(dataset)) if dataset[i][1] == 3]

dataset = torch.utils.data.Subset(dataset, ind)

# Wrap the combined dataset with DataLoader for batching
dataloader = torch.utils.data.DataLoader(dataset = dataset, batch_size=64, shuffle=True)

In [4]:
import matplotlib.pyplot as plt

# Get a batch of images and labels
images, labels = next(iter(dataloader))

# # Create a grid of images
# grid = utils.make_grid(images[:16], nrow=4, padding=2, normalize=True)

# # Display the grid
# plt.figure(figsize=(8, 8))
# plt.imshow(grid.permute(1, 2, 0))  # Convert from (C, H, W) to (H, W, C) for plotting
# plt.axis('off')
# plt.show()

: 

In [ ]:
plt.imshow(images[0].squeeze(-1), cmap = 'gray')

TypeError: Invalid shape (1, 28, 28) for image data